In [9]:
import pandas as pd
import numpy as np
import re
import os
import warnings
import string
import nltk
from time import time
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn import metrics
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec, FastText
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.metrics.pairwise import euclidean_distances
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from collections import defaultdict
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SUCI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SUCI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#Feature Extraction and Clustering using experiment I

In [11]:
nmfile = 'data-after-preprocess-experiment1'
df = pd.read_csv(nmfile+'.csv')
nmkolom = df.columns

print (nmkolom)

kolomproduk = 'namaproduk'
kolomdeskripsi = 'desc'

nmproduk = df[kolomproduk].tolist()
deskripsi = df[kolomdeskripsi].tolist()

kategori = df['kategori'].tolist()
subkategori = df['subkategori'].tolist()

rawdataset = []

def get_dataset():
    global df
    df = df.reset_index()  # make sure indexes pair with number of rows
    testdata = ''
    i = 0
    iambil = 1650
    sentences = []

    for index, row in df.iterrows():
        sebaris1 = row[kolomproduk]
        if (pd.isna(sebaris1)):
            sebaris1 = ' '

        sebaris2 = row[kolomdeskripsi]
        if (pd.isna(sebaris2)):
            sebaris2 = ' '

        kalimat = sebaris1 + ' ' + sebaris2
        if (i==iambil):
            testdata = kalimat
            
        sentences.append(kalimat)
        i = i + 1

    return (testdata, sentences)

testdata, dataset_data = get_dataset()


labels = kategori
unique_labels, category_sizes = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]

print(f"{len(dataset_data)} documents - {true_k} categories")
print (unique_labels)


evaluations = []
evaluations_std = []


def fit_and_evaluate(km, X, kateg_sizes, name=None, n_runs=5):
    name = km._class.name_ if name is None else name

    train_times = []
    scores = defaultdict(list)
    seed_sse_terkecil = 0
    sse_lama = 999999
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        t0 = time()
        km.fit(X)
        train_times.append(time() - t0)
        cluster_ids, cluster_sizes = np.unique(km.labels_, return_counts=True)
        sse = 0
        for i in range(len(kateg_sizes)):
            sse = sse + pow(cluster_sizes[i] - kateg_sizes[i], 2)
            
        if (sse<sse_lama):
            seed_sse_terkecil = seed 
        sse_lama = sse 
        
        print(f"Number of elements asigned to each cluster: {cluster_sizes}, sse= {sse}, seed= {seed}") 

        scores["Homogeneity"].append(metrics.homogeneity_score(labels, km.labels_))
        scores["Completeness"].append(metrics.completeness_score(labels, km.labels_))
        scores["V-measure"].append(metrics.v_measure_score(labels, km.labels_))
        scores["Adjusted Rand-Index"].append(
            metrics.adjusted_rand_score(labels, km.labels_)
        )
        scores["Silhouette Coefficient"].append(
            metrics.silhouette_score(X, km.labels_, sample_size=2000)
        )
    train_times = np.asarray(train_times)

    print(f"clustering done in {train_times.mean():.2f} ± {train_times.std():.2f} s ")
    evaluation = {
        "estimator": name,
        "train_time": train_times.mean(),
    }
    evaluation_std = {
        "estimator": name,
        "train_time": train_times.std(),
    }
    for score_name, score_values in scores.items():
        mean_score, std_score = np.mean(score_values), np.std(score_values)
        print(f"{score_name}: {mean_score:.3f} ± {std_score:.3f}")
        evaluation[score_name] = mean_score
        evaluation_std[score_name] = std_score
    evaluations.append(evaluation)
    evaluations_std.append(evaluation_std)
    return seed_sse_terkecil

vectorizer = TfidfVectorizer(max_df=0.5, min_df=5, stop_words="english")
t0 = time()
X_tfidf = vectorizer.fit_transform(dataset_data)

print(f"vectorization done in {time() - t0:.3f} s")
print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

os.environ["OMP_NUM_THREADS"] = '8'

print ("\n*** KMeans with LSA on tf-idf vectors:")
print(
    "True number of documents in each category according to the class labels: "
    f"{category_sizes}"
)

print ("Performing dimensionality reduction using LSA ...")
lsa = make_pipeline(TruncatedSVD(n_components=100), Normalizer(copy=False))
t0 = time()
X_lsa = lsa.fit_transform(X_tfidf)
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"LSA done in {time() - t0:.3f} s")
print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

kmeans = KMeans(n_clusters=true_k, max_iter=100, n_init=5)

seed_pilih = fit_and_evaluate(kmeans, X_lsa, category_sizes, name="KMeans\non tf-idf vectors")


kmeans = KMeans(n_clusters=true_k, max_iter=100, n_init=5, random_state=seed_pilih)
kmeans.fit(X_lsa)
centroids = kmeans.cluster_centers_
cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
labels = kmeans.labels_
sse = 0
for i in range(len(category_sizes)):
    sse = sse + pow(cluster_sizes[i] - category_sizes[i], 2)

print(f"Number of elements asigned to each cluster: {cluster_sizes}, sse= {sse}")

def distance_from_center(X, centroids):

	distance=euclidean_distances(X, np.array(centroids) )

	hasil = np.min(distance, axis=1)
	print (distance.shape)
	print (hasil.shape)

	return hasil   
    
euc_res=distance_from_center(X_lsa, centroids)

# normalize the result  
normlaized_res= euc_res  
dfx = pd.DataFrame(normlaized_res, columns=columnName)
dfx['cluster'] = labels
dfx['kategori'] = kategori

print (normlaized_res.shape)

jumlahNpertama = 50

for clusterx in cluster_ids:
    dfy = dfx[dfx['cluster'] == clusterx]
    dfz = dfy.sort_values(by=[nmkol], ascending=False)
    # misal ditampilkan top ten outliers
    outliers_idx = dfz.head(jumlahNpertama).index
    #outliers = dfx[dfx.index.isin(outliers_idx)]
    for idx in outliers_idx:
        distance  = dfx.iloc[idx][nmkol]  
        kategorix = df.iloc[idx]['kategori']   
        nmproduk = df.iloc[idx]['namaproduk']
        nmproduk = nmproduk[0:30]
        
dfx.to_csv(nmfile+'-cluster-LSA.csv')

Index(['namaproduk', 'desc', 'harga', 'terjual', 'rating', 'namatoko',
       'kategori', 'subkategori', 'marketplace'],
      dtype='object')
1959 documents - 5 categories
['ATK' 'Elektronik' 'Kesehatan' 'Kosmetik' 'Pakaian']
vectorization done in 0.118 s
n_samples: 1959, n_features: 1922

*** KMeans with LSA on tf-idf vectors:
True number of documents in each category according to the class labels: [391 392 380 397 399]
Performing dimensionality reduction using LSA ...
LSA done in 0.169 s
Explained variance of the SVD step: 59.3%
Number of elements asigned to each cluster: [717 270 417 274 281], sse= 151582, seed= 0
Number of elements asigned to each cluster: [ 95 271 200 940 453], sse= 432422, seed= 1
Number of elements asigned to each cluster: [274 694 191 154 646], sse= 260672, seed= 2
Number of elements asigned to each cluster: [196 491 849 271 152], sse= 344672, seed= 3
Number of elements asigned to each cluster: [466 199 287 522 485], sse= 74544, seed= 4
clustering done in 0.10

In [ ]:
#Feature Extraction and Clustering using experiment II

In [12]:
nmfile = 'data-after-preprocess-experiment2'
df = pd.read_csv(nmfile+'.csv')
nmkolom = df.columns

print (nmkolom)

kolomproduk = 'namaproduk'
kolomdeskripsi = 'desc'

nmproduk = df[kolomproduk].tolist()
deskripsi = df[kolomdeskripsi].tolist()

kategori = df['kategori'].tolist()
subkategori = df['subkategori'].tolist()

rawdataset = []

def get_dataset():
    global df
    df = df.reset_index()  # make sure indexes pair with number of rows
    testdata = ''
    i = 0
    iambil = 1650
    sentences = []

    for index, row in df.iterrows():
        sebaris1 = row[kolomproduk]
        if (pd.isna(sebaris1)):
            sebaris1 = ' '

        sebaris2 = row[kolomdeskripsi]
        if (pd.isna(sebaris2)):
            sebaris2 = ' '

        kalimat = sebaris1 + ' ' + sebaris2
        if (i==iambil):
            testdata = kalimat
            
        sentences.append(kalimat)
        i = i + 1

    return (testdata, sentences)

testdata, dataset_data = get_dataset()


labels = kategori
unique_labels, category_sizes = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]

print(f"{len(dataset_data)} documents - {true_k} categories")
print (unique_labels)


evaluations = []
evaluations_std = []


def fit_and_evaluate(km, X, kateg_sizes, name=None, n_runs=5):
    name = km._class.name_ if name is None else name

    train_times = []
    scores = defaultdict(list)
    seed_sse_terkecil = 0
    sse_lama = 999999
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        t0 = time()
        km.fit(X)
        train_times.append(time() - t0)
        cluster_ids, cluster_sizes = np.unique(km.labels_, return_counts=True)
        sse = 0
        for i in range(len(kateg_sizes)):
            sse = sse + pow(cluster_sizes[i] - kateg_sizes[i], 2)
            
        if (sse<sse_lama):
            seed_sse_terkecil = seed 
        sse_lama = sse 
        
        print(f"Number of elements asigned to each cluster: {cluster_sizes}, sse= {sse}, seed= {seed}") 

        scores["Homogeneity"].append(metrics.homogeneity_score(labels, km.labels_))
        scores["Completeness"].append(metrics.completeness_score(labels, km.labels_))
        scores["V-measure"].append(metrics.v_measure_score(labels, km.labels_))
        scores["Adjusted Rand-Index"].append(
            metrics.adjusted_rand_score(labels, km.labels_)
        )
        scores["Silhouette Coefficient"].append(
            metrics.silhouette_score(X, km.labels_, sample_size=2000)
        )
    train_times = np.asarray(train_times)

    print(f"clustering done in {train_times.mean():.2f} ± {train_times.std():.2f} s ")
    evaluation = {
        "estimator": name,
        "train_time": train_times.mean(),
    }
    evaluation_std = {
        "estimator": name,
        "train_time": train_times.std(),
    }
    for score_name, score_values in scores.items():
        mean_score, std_score = np.mean(score_values), np.std(score_values)
        print(f"{score_name}: {mean_score:.3f} ± {std_score:.3f}")
        evaluation[score_name] = mean_score
        evaluation_std[score_name] = std_score
    evaluations.append(evaluation)
    evaluations_std.append(evaluation_std)
    return seed_sse_terkecil

vectorizer = TfidfVectorizer(max_df=0.5, min_df=5, stop_words="english")
t0 = time()
X_tfidf = vectorizer.fit_transform(dataset_data)

print(f"vectorization done in {time() - t0:.3f} s")
print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

os.environ["OMP_NUM_THREADS"] = '8'

print ("\n*** KMeans with LSA on tf-idf vectors:")
print(
    "True number of documents in each category according to the class labels: "
    f"{category_sizes}"
)

print ("Performing dimensionality reduction using LSA ...")
lsa = make_pipeline(TruncatedSVD(n_components=100), Normalizer(copy=False))
t0 = time()
X_lsa = lsa.fit_transform(X_tfidf)
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"LSA done in {time() - t0:.3f} s")
print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

kmeans = KMeans(n_clusters=true_k, max_iter=100, n_init=5)

seed_pilih = fit_and_evaluate(kmeans, X_lsa, category_sizes, name="KMeans\non tf-idf vectors")


kmeans = KMeans(n_clusters=true_k, max_iter=100, n_init=5, random_state=seed_pilih)
kmeans.fit(X_lsa)
centroids = kmeans.cluster_centers_
cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
labels = kmeans.labels_
sse = 0
for i in range(len(category_sizes)):
    sse = sse + pow(cluster_sizes[i] - category_sizes[i], 2)

print(f"Number of elements asigned to each cluster: {cluster_sizes}, sse= {sse}")

def distance_from_center(X, centroids):

	distance=euclidean_distances(X, np.array(centroids) )

	hasil = np.min(distance, axis=1)
	print (distance.shape)
	print (hasil.shape)

	return hasil   
    
euc_res=distance_from_center(X_lsa, centroids)

# normalize the result  
normlaized_res= euc_res  
dfx = pd.DataFrame(normlaized_res, columns=columnName)
dfx['cluster'] = labels
dfx['kategori'] = kategori

print (normlaized_res.shape)

jumlahNpertama = 50

for clusterx in cluster_ids:
    dfy = dfx[dfx['cluster'] == clusterx]
    dfz = dfy.sort_values(by=[nmkol], ascending=False)
    # misal ditampilkan top ten outliers
    outliers_idx = dfz.head(jumlahNpertama).index
    #outliers = dfx[dfx.index.isin(outliers_idx)]
    for idx in outliers_idx:
        distance  = dfx.iloc[idx][nmkol]  
        kategorix = df.iloc[idx]['kategori']   
        nmproduk = df.iloc[idx]['namaproduk']
        nmproduk = nmproduk[0:30]
        
dfx.to_csv(nmfile+'-cluster-LSA.csv')

Index(['namaproduk', 'desc', 'harga', 'terjual', 'rating', 'namatoko',
       'kategori', 'subkategori', 'marketplace'],
      dtype='object')
1984 documents - 5 categories
['ATK' 'Elektronik' 'Kesehatan' 'Kosmetik' 'Pakaian']
vectorization done in 0.179 s
n_samples: 1984, n_features: 2942

*** KMeans with LSA on tf-idf vectors:
True number of documents in each category according to the class labels: [394 398 395 398 399]
Performing dimensionality reduction using LSA ...
LSA done in 0.249 s
Explained variance of the SVD step: 50.2%
Number of elements asigned to each cluster: [390  98 293 808 395], sse= 268536, seed= 0
Number of elements asigned to each cluster: [286 640 284 390 384], sse= 82838, seed= 1
Number of elements asigned to each cluster: [289 620 583  98 394], sse= 185678, seed= 2
Number of elements asigned to each cluster: [275 382 941 287  99], sse= 414854, seed= 3
Number of elements asigned to each cluster: [199 701 701 196 187], sse= 309218, seed= 4
clustering done in 0.10

In [ ]:
#Feature Extraction and Clustering using experiment III

In [14]:
nmfile = 'datas-after-preprocess-experiment3'
df = pd.read_csv(nmfile+'.csv')
nmkolom = df.columns

print (nmkolom)

kolomproduk = 'namaproduk'
kolomdeskripsi = 'desc'

nmproduk = df[kolomproduk].tolist()
deskripsi = df[kolomdeskripsi].tolist()

kategori = df['kategori'].tolist()
subkategori = df['subkategori'].tolist()

rawdataset = []

def get_dataset():
    global df
    df = df.reset_index()  # make sure indexes pair with number of rows
    testdata = ''
    i = 0
    iambil = 1650
    sentences = []

    for index, row in df.iterrows():
        sebaris1 = row[kolomproduk]
        if (pd.isna(sebaris1)):
            sebaris1 = ' '

        sebaris2 = row[kolomdeskripsi]
        if (pd.isna(sebaris2)):
            sebaris2 = ' '

        kalimat = sebaris1 + ' ' + sebaris2
        if (i==iambil):
            testdata = kalimat
            
        sentences.append(kalimat)
        i = i + 1

    return (testdata, sentences)

testdata, dataset_data = get_dataset()


labels = kategori
unique_labels, category_sizes = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]

print(f"{len(dataset_data)} documents - {true_k} categories")
print (unique_labels)


evaluations = []
evaluations_std = []


def fit_and_evaluate(km, X, kateg_sizes, name=None, n_runs=5):
    name = km._class.name_ if name is None else name

    train_times = []
    scores = defaultdict(list)
    seed_sse_terkecil = 0
    sse_lama = 999999
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        t0 = time()
        km.fit(X)
        train_times.append(time() - t0)
        cluster_ids, cluster_sizes = np.unique(km.labels_, return_counts=True)
        sse = 0
        for i in range(len(kateg_sizes)):
            sse = sse + pow(cluster_sizes[i] - kateg_sizes[i], 2)
            
        if (sse<sse_lama):
            seed_sse_terkecil = seed 
        sse_lama = sse 
        
        print(f"Number of elements asigned to each cluster: {cluster_sizes}, sse= {sse}, seed= {seed}") 

        scores["Homogeneity"].append(metrics.homogeneity_score(labels, km.labels_))
        scores["Completeness"].append(metrics.completeness_score(labels, km.labels_))
        scores["V-measure"].append(metrics.v_measure_score(labels, km.labels_))
        scores["Adjusted Rand-Index"].append(
            metrics.adjusted_rand_score(labels, km.labels_)
        )
        scores["Silhouette Coefficient"].append(
            metrics.silhouette_score(X, km.labels_, sample_size=2000)
        )
    train_times = np.asarray(train_times)

    print(f"clustering done in {train_times.mean():.2f} ± {train_times.std():.2f} s ")
    evaluation = {
        "estimator": name,
        "train_time": train_times.mean(),
    }
    evaluation_std = {
        "estimator": name,
        "train_time": train_times.std(),
    }
    for score_name, score_values in scores.items():
        mean_score, std_score = np.mean(score_values), np.std(score_values)
        print(f"{score_name}: {mean_score:.3f} ± {std_score:.3f}")
        evaluation[score_name] = mean_score
        evaluation_std[score_name] = std_score
    evaluations.append(evaluation)
    evaluations_std.append(evaluation_std)
    return seed_sse_terkecil

vectorizer = TfidfVectorizer(max_df=0.5, min_df=5, stop_words="english")
t0 = time()
X_tfidf = vectorizer.fit_transform(dataset_data)

print(f"vectorization done in {time() - t0:.3f} s")
print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

os.environ["OMP_NUM_THREADS"] = '8'

print ("\n*** KMeans with LSA on tf-idf vectors:")
print(
    "True number of documents in each category according to the class labels: "
    f"{category_sizes}"
)

print ("Performing dimensionality reduction using LSA ...")
lsa = make_pipeline(TruncatedSVD(n_components=100), Normalizer(copy=False))
t0 = time()
X_lsa = lsa.fit_transform(X_tfidf)
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"LSA done in {time() - t0:.3f} s")
print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

kmeans = KMeans(n_clusters=true_k, max_iter=100, n_init=5)

seed_pilih = fit_and_evaluate(kmeans, X_lsa, category_sizes, name="KMeans\non tf-idf vectors")


kmeans = KMeans(n_clusters=true_k, max_iter=100, n_init=5, random_state=seed_pilih)
kmeans.fit(X_lsa)
centroids = kmeans.cluster_centers_
cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
labels = kmeans.labels_
sse = 0
for i in range(len(category_sizes)):
    sse = sse + pow(cluster_sizes[i] - category_sizes[i], 2)

print(f"Number of elements asigned to each cluster: {cluster_sizes}, sse= {sse}")

def distance_from_center(X, centroids):

	distance=euclidean_distances(X, np.array(centroids) )

	hasil = np.min(distance, axis=1)
	print (distance.shape)
	print (hasil.shape)

	return hasil   
    
euc_res=distance_from_center(X_lsa, centroids)

# normalize the result  
normlaized_res= euc_res 
dfx = pd.DataFrame(normlaized_res, columns=columnName)
dfx['cluster'] = labels
dfx['kategori'] = kategori

print (normlaized_res.shape)

jumlahNpertama = 50

for clusterx in cluster_ids:
    dfy = dfx[dfx['cluster'] == clusterx]
    dfz = dfy.sort_values(by=[nmkol], ascending=False)
    # misal ditampilkan top ten outliers
    outliers_idx = dfz.head(jumlahNpertama).index
    #outliers = dfx[dfx.index.isin(outliers_idx)]
    for idx in outliers_idx:
        distance  = dfx.iloc[idx][nmkol]  
        kategorix = df.iloc[idx]['kategori']   
        nmproduk = df.iloc[idx]['namaproduk']
        nmproduk = nmproduk[0:30]
        
dfx.to_csv(nmfile+'-cluster-LSA.csv')

Index(['namaproduk', 'desc', 'harga', 'terjual', 'rating', 'namatoko',
       'kategori', 'subkategori', 'marketplace'],
      dtype='object')
2000 documents - 5 categories
['ATK' 'Elektronik' 'Kesehatan' 'Kosmetik' 'Pakaian']
vectorization done in 0.418 s
n_samples: 2000, n_features: 4259

*** KMeans with LSA on tf-idf vectors:
True number of documents in each category according to the class labels: [400 400 400 400 400]
Performing dimensionality reduction using LSA ...
LSA done in 0.489 s
Explained variance of the SVD step: 46.5%
Number of elements asigned to each cluster: [266 351 293 673 417], sse= 106624, seed= 0
Number of elements asigned to each cluster: [  99  480  193 1128  100], sse= 759834, seed= 1
Number of elements asigned to each cluster: [760 435 513 192 100], sse= 276858, seed= 2
Number of elements asigned to each cluster: [625 485 294 388 208], sse= 106094, seed= 3
Number of elements asigned to each cluster: [193 473 381 853 100], sse= 343748, seed= 4
clustering done i